In [1]:
import os

from setuptools.archive_util import unpack_zipfile

In [2]:
%pwd

'C:\\Users\\RameezRassdeen\\Documents\\wine-quality-prediction-webapp\\notebooks'

In [3]:
os.chdir('..')
%pwd

'C:\\Users\\RameezRassdeen\\Documents\\wine-quality-prediction-webapp'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    """
    Configuration for data ingestion.
    """
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from wine_quality_prediction.constants import *
from wine_quality_prediction.utils.common import read_yaml, create_directory

In [6]:
class ConfigurationManager:
    def __init__(self,
            config_file_path: Path = CONFIG_FILE_PATH,
            params_file_path: Path = PARAMS_FILE_PATH,
            schema_file_path: Path = SCHEMA_FILE_PATH):

            self.config = read_yaml(config_file_path)
            self.params = read_yaml(params_file_path)
            self.schema = read_yaml(schema_file_path)

            create_directory([self.config.artifacts_root], verbose=True)


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directory([config.root_dir], verbose=True)

        data_ingestion_config_ = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config_

In [7]:
import os
import urllib.request as request
import zipfile
from wine_quality_prediction import logger
from wine_quality_prediction.utils.common import get_size
from pathlib import Path

In [8]:
# Defining the DataIngestion class
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self, headers=None):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"Data downloaded from the source URL: %s with the following info: \n{headers}", self.config.source_URL)
        else:
            logger.info(f"Data already exists at the path: %s with the following size info: \n{get_size(Path(self.config.local_data_file))}", self.config.local_data_file)

    def extract_zip(self):
        unzip_path = self.config.unzip_dir
        if not os.path.exists(unzip_path):
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
            logger.info("Data extracted to the path: %s", unzip_path)
            logger.info("Size of the extracted data:")

        else:
            logger.info("Data already extracted at the path: %s", unzip_path)
            get_size(unzip_path)

In [9]:
try:
    config_manager = ConfigurationManager()
    data_ingestion_config = config_manager.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip()
except Exception as e:
    raise e

[2025-01-21 23:03:31,623 - wine_quality_prediction - INFO - YAML file loaded successfully from the path: config\config.yaml]
[2025-01-21 23:03:31,624 - wine_quality_prediction - INFO - YAML file loaded successfully from the path: params.yaml]
[2025-01-21 23:03:31,626 - wine_quality_prediction - INFO - YAML file loaded successfully from the path: schema.yaml]
[2025-01-21 23:03:31,627 - wine_quality_prediction - INFO - Directory already exists at the path: artifacts]
[2025-01-21 23:03:31,629 - wine_quality_prediction - INFO - Directory already exists at the path: artifacts/data_ingestion]
[2025-01-21 23:03:31,631 - wine_quality_prediction - INFO - Data already exists at the path: artifacts/data_ingestion/wine_quality_data.zip with the following size info: 
24.6 KB]
[2025-01-21 23:03:31,631 - wine_quality_prediction - INFO - Data already extracted at the path: artifacts/data_ingestion/wine_quality_data]
